In [30]:
import geopandas as gpd
from pathlib import Path
import os

In [31]:

data_path = Path(r'c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample')
basins_path = data_path / 'DERP_sample_basins.geojson'
basins = gpd.read_file(basins_path)
basins.head(3)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,...,ID_0,ISO,NAME_0,ID_1,NAME_1,NL_NAME_1,VARNAME_1,TYPE_1,ENGTYPE_1,geometry
0,2080420340,2080419180,2080023010,2080023010,447.9,447.9,34.2,109585.3,23263010,0,...,86,DEU,Germany,11,Rheinland-Pfalz,None,Rhineland-Palatinate,Land,State,"MULTIPOLYGON (((7.59583 50.30833, 7.59583 50.3..."
1,2080421770,2080420340,2080023010,2080023010,454.5,454.5,425.3,103638.1,23263030,0,...,86,DEU,Germany,11,Rheinland-Pfalz,None,Rhineland-Palatinate,Land,State,"MULTIPOLYGON (((7.72574 49.99653, 7.72434 50.0..."
2,2080428500,2080421770,2080023010,2080023010,500.1,500.1,69.3,103003.2,23263050,0,...,86,DEU,Germany,11,Rheinland-Pfalz,None,Rhineland-Palatinate,Land,State,"MULTIPOLYGON (((7.84105 49.96466, 7.83812 49.9..."


In [32]:
scenarios=['H', 'M', 'L']
for scenario in scenarios:
    for hybas_id in basins['HYBAS_ID']:
        flood_event_map_path = fr'C:\Data\Floods\Germany\basin_intersections\DERP_RW_{scenario}_4326_hybas_intersections\flood_DERP_RW_{scenario}_4326_{hybas_id}.geojson'
        output_path = data_path / fr'Floods\Germany\basin_intersections\flood_DERP_RW_{scenario}_4326_{hybas_id}.geojson'
        print(output_path)
        # copy the file to the output path creating any necessary directories
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        os.system(f'copy "{flood_event_map_path}" "{output_path}"')


c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample\Floods\Germany\basin_intersections\flood_DERP_RW_H_4326_2080420340.geojson
c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample\Floods\Germany\basin_intersections\flood_DERP_RW_H_4326_2080421770.geojson
c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample\Floods\Germany\basin_intersections\flood_DERP_RW_H_4326_2080428500.geojson
c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample\Floods\Germany\basin_intersections\flood_DERP_RW_H_4326_2080430240.geojson
c:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\sample\Floods\Germany\basin_intersections\flood_DERP_RW_H_4326_2080430320.geo

In [ ]:
# Create visualisation for the basin and the discharge points
from lonboard import Map, PolygonLayer, PathLayer

layer_basin = None#PolygonLayer.from_geopandas(basins_filtered,
    # get_fill_color=miraca_colors['grey_100'],
    # get_line_color=miraca_colors['green_900'], get_line_width=100,
    # auto_highlight=False,
    # filled=False)

# Flood return period: H for frequent(RP10-20), M for 100 year return period (RP100) and L for extreme (RP2000)
return_period_str='L'
basin_vis_str='2080430320'
# Generate flood layers and protection layers for visualization
flood_plot_path=r'Floods\Germany\basin_intersections\flood_DERP_RW_L_4326_2080430320.geojson'
flood_m = data_path / flood_plot_path
flood_gdf=gpd.read_file(flood_m)
layers_flood=[]
f_area_colors = {1:'blue', 3:'green'}
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            layers_flood.append(PolygonLayer.from_geopandas(subset_gdf, 
                                                            get_fill_color=miraca_colors[color_key], 
                                                            opacity=0.5, 
                                                            stroked=False))

assets_no_buffer = assets.copy()
#drop the buffered geometry from the assets
# assets_no_buffer = assets_no_buffer.drop(columns='buffered')
layer_rail = PathLayer.from_geopandas(assets_no_buffer, width_min_pixels=0.8)

layers=[]
if layer_basin is not None:
    layers.append(layer_basin)
else:
    print('No basin layer')
if layers_flood is not None:
    layers.extend(layers_flood)
else:
    print('No flood layers')
if layer_rail is not None:
    layers.append(layer_rail)
else:
    print('No rail layers')

Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'

m = Map(layers, show_tooltip=True, basemap_style=Voyager) 

m
